### Importing Libraries

In [1]:
! pip install -q split-folders

In [2]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sn
import sklearn.metrics


import splitfolders

### Loading Dataset & Data Augmentation

In [3]:
! mkdir data

In [4]:
splitfolders.ratio("../input/boat-images", output="./data", seed=46, ratio=(.8, .2), group_prefix=None) # default values

In [5]:
transform = transforms.Compose([
    transforms.Resize([100,100]),
    transforms.RandomRotation(degrees=[-30,30]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform2 = transforms.Compose([
    transforms.Resize([100,100]),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [6]:
data_dir = './data/train'
classes = os.listdir(data_dir)
print(classes)
print("Total Class:",len(classes))

In [7]:
test_ds = torchvision.datasets.ImageFolder('./data/val', transform=transform2)
train_ds = torchvision.datasets.ImageFolder('./data/train', transform=transform)

In [8]:
print(len(train_ds))

In [9]:
print(len(test_ds))

In [10]:
train_ds.classes

In [11]:
# view one image shape of the dataset.
img, label = train_ds[10]
print(img.shape)

### Make Dataset Iterable

In [21]:
batch_size = 16

num_epochs = 150

In [13]:
train_loader = torch.utils.data.DataLoader(dataset = train_ds, batch_size=batch_size, shuffle=True )
test_loader = torch.utils.data.DataLoader(dataset = test_ds, batch_size=batch_size, shuffle=False)

### Create Model Class | CNN Model 1

In [14]:
class CNNModelOne(nn.Module):
    def __init__(self):
        super(CNNModelOne, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()
        
        # Max pool 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 
        self.fc1 = nn.Linear(128 * 12 * 12, 1024) 
        self.lrelu1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.40)
        
        self.fc2 = nn.Linear(1024, 512)
        self.lrelu2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.30)
        
        self.fc3 = nn.Linear(512, 256)
        self.lrelu3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.25)
        
        self.fc4 = nn.Linear(256, 9)
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Convolution 3 
        out = self.cnn3(out)
        out = self.relu3(out)
        
        # Max pool 3 
        out = self.maxpool3(out)
        
        # Flatten
        out = torch.flatten(out, 1) # flatten all dimensions except the batch dimension

        # Linear function (readout)
        out = self.fc1(out)
        out = self.lrelu1(out)
        out = self.drop1(out)
        
        out = self.fc2(out)
        out = self.lrelu2(out)
        out = self.drop2(out)
        
        out = self.fc3(out)
        out = self.lrelu3(out)
        out = self.drop3(out)
        
        out = self.fc4(out)
        
        return out

### Instantiate Model Class

In [15]:
model = CNNModelOne()

In [16]:
model = model.cuda()

### Instantiate Loss Class

In [17]:
criterion = nn.CrossEntropyLoss()

### Instantiate Optimizer Class

In [18]:
learning_rate = 0.0001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

Parameters In-Depth

In [19]:
print(model)

In [20]:
print(model.parameters())

print(len(list(model.parameters())))

### Train Model


In [22]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images.cuda())
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels.cuda())
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
 
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in test_loader:
         # Load images
        images = images.requires_grad_()
                
        # Forward pass only to get logits/output
        outputs = model(images.cuda())
                
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
                
        labels = labels.cuda() 
        # Total number of labels
        total += labels.size(0)
                
        # Total correct predictions
        correct += (predicted == labels).sum()
            
    accuracy = 100 * correct / total
            
    # Print Loss
    print('Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}'.format(epoch, iter, loss.item(), accuracy))